## UPLOAD DATAFRAMES AND DEFINE FUNCTIONS

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import re
from urllib.parse import quote, unquote, urlunparse, urlparse
import openpyxl
import uuid
import ast

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-site-isolation-trials")

In [43]:
ranksnew = pd.read_stata("ranks_wwii_new.dta")
generals4041 = pd.read_csv("generals40-41.csv")
repressions_main_table = pd.read_csv("Repressions.csv") # старая база данных
generals45 = pd.read_excel("generals40-45.xls")
generals4041 = generals4041.fillna('')
generals4041["name_title"] = generals4041["Фамилия"] + " " + generals4041["Имя"] + " " + generals4041["Отчество"] + " " + generals4041["Воинское звание"]
generals4041["name"] = generals4041["Фамилия"] + " " + generals4041["Имя"] + " " + generals4041["Отчество"]
list_names_titles_4041 = list(generals4041["name_title"])
list_names_4041 = list(generals4041["name"])
list_names_4041 = [value for value in list_names_4041 if value != '  ']
generals4041.head(5)

,Фамилия,Имя,Отчество,Воинское звание,Дата,Номер,name_title,name
0,АБАНЬКИН,Павел,Сергеевич,Контр-адмирал,5/21/41,1394.0,АБАНЬКИН Павел Сергеевич Контр-адмирал,АБАНЬКИН Павел Сергеевич
1,АБРАМИДЗЕ,Павел,Ивлианович,Генерал-майор,6/4/40,945.0,АБРАМИДЗЕ Павел Ивлианович Генерал-майор,АБРАМИДЗЕ Павел Ивлианович
2,АБРАМОВ,Николай,Осипович,Контр-адмирал,6/4/40,946.0,АБРАМОВ Николай Осипович Контр-адмирал,АБРАМОВ Николай Осипович
3,АБСАЛЯМОВ,Манзакир,,Генерал-майор,6/4/40,945.0,АБСАЛЯМОВ Манзакир Генерал-майор,АБСАЛЯМОВ Манзакир
4,АВВАКУМОВ,Яков,Александрович,Генерал-майор,6/4/40,945.0,АВВАКУМОВ Яков Александрович Генерал-майор,АВВАКУМОВ Яков Александрович


In [36]:
def get_PM_link_to_general(fullname, birth_year):
    print(fullname)
    # Set up the WebDriver (ensure you have the correct path to your WebDriver)
    # driver_path = '/Users/elenasmyslovskih/Desktop/chromedriver_mac64/chromedriver'
    service = webdriver.ChromeService()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(600) 
    
    # Define the search parameters
    base_url = "https://pamyat-naroda.ru/heroes/?adv_search=y"
    if len(fullname.split(' ')) == 3:
        last_name = fullname.split(' ')[0].lower().capitalize()
        first_name = fullname.split(' ')[1]
        middle_name = fullname.split(' ')[2]
    else:
        last_name = fullname.split(' ')[0].lower().capitalize()
        first_name = fullname.split(' ')[1]
        middle_name = ''
    
    # Construct the search URL with the parameters
    search_url = f"{base_url}&last_name={last_name}&first_name={first_name}&middle_name={middle_name}&date_birth_from={birth_year}&group=all&types=pamyat_commander:nagrady_nagrad_doc:nagrady_uchet_kartoteka:nagrady_ubilein_kartoteka:pdv_kart_in:pdv_kart_in_inostranec:pamyat_voenkomat:potery_vpp:pamyat_zsp_parts:kld_ran:kld_bolezn:kld_polit:kld_upk:kld_vmf:kld_partizan:potery_doneseniya_o_poteryah:potery_gospitali:potery_utochenie_poter:potery_spiski_zahoroneniy:potery_voennoplen:potery_iskluchenie_iz_spiskov:potery_kartoteki:potery_rvk_extra:potery_isp_extra:same_doroga:same_rvk:same_guk:potery_knigi_pamyati&page=1&grouppersons=1"
    
    # Open the search URL
    driver.get(search_url)
    
    # Wait for the results to load (you may need to adjust the wait condition)
    #WebDriverWait(driver, 2).until(
        #EC.presence_of_element_located((By.CLASS_NAME, "result-item"))
    #)
    
    # Get the page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find link to the general from list of links
    descriptions = soup.find_all('p', class_='heroes-list-item-info')
    links = soup.find_all('a', class_='heroes-list-item-name')
    descriptions_list = refs_list = [description.text for description in descriptions]
    links_list = [link['href'] for link in links]
    print(descriptions_list)

    link_to_general = ''

    
    if len(descriptions_list) > 0:
        for description in descriptions_list:
            if ("Генерал" in description) or ("Маршал" in description) or ("Полковник" in description):
                general_idx = descriptions_list.index(description)
                link_to_general = links_list[general_idx]
        print('resulting link -----------------------------------')
        print(link_to_general)
    else:
        print('Descriptions not found')
    return link_to_general

In [ ]:
#df8['link_PN'] = df8.apply(lambda row: get_PM_link_to_general(row['fullname'], row['birth_year']), axis=1)

In [24]:
#df8.to_excel('intermediate_df8_with_PN.xls', index=False, engine='openpyxl')

In [45]:
url1 = 'https://pamyat-naroda.ru/heroes/person-hero123940112/?backurl=%2Fheroes%2F%3Fadv_search%3Dy%26last_name%3D%D0%91%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D1%8F%D0%BD%26first_name%3D%D0%98%D0%B2%D0%B0%D0%BD%26middle_name%3D%D0%A5%D1%80%D0%B8%D1%81%D1%82%D0%BE%D1%84%D0%BE%D1%80%D0%BE%D0%B2%D0%B8%D1%87%26group%3Dall%26types%3Dpamyat_commander%3Anagrady_nagrad_doc%3Anagrady_uchet_kartoteka%3Anagrady_ubilein_kartoteka%3Apdv_kart_in%3Apdv_kart_in_inostranec%3Apamyat_voenkomat%3Apotery_vpp%3Apamyat_zsp_parts%3Akld_ran%3Akld_bolezn%3Akld_polit%3Akld_upk%3Akld_vmf%3Akld_partizan%3Apotery_doneseniya_o_poteryah%3Apotery_gospitali%3Apotery_utochenie_poter%3Apotery_spiski_zahoroneniy%3Apotery_voennoplen%3Apotery_iskluchenie_iz_spiskov%3Apotery_kartoteki%3Apotery_rvk_extra%3Apotery_isp_extra%3Asame_doroga%3Asame_rvk%3Asame_guk%3Apotery_knigi_pamyati%26page%3D1%26grouppersons%3D1&'

In [46]:
def get_awards_by_date(url):
    print(url)

    if pd.isnull(url):
        print("URL is missing.")
        return []

    else:
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--disable-site-isolation-trials")
        
        service = webdriver.ChromeService()
        driver = webdriver.Chrome(service=service, options=chrome_options)
        #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        driver.set_page_load_timeout(1200)
        
        try:
            driver.get(url)
    
            # Wait for the necessary element to load
            driver.implicitly_wait(10)
            
            # Get the page source and parse it with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            doc_sections = soup.find_all('div', class_='js-hero-card-doc hero-card-docs-item hero-card-docs-item_reward')
        
            if not doc_sections:
                return []

            awards = []
            
            for section in doc_sections:
                # Find the <b> tag containing "Наименование награды:"
                tag = section.find('b', text='Наименование награды:')
                
                # Get the award name from the next sibling of the <b> tag
                award_name = tag.next_sibling.strip() if tag else None
                
                # Find the <b> tag containing "Дата документа:"
                b_tag = section.find('b', text='Дата документа:')
                if b_tag:
                    # Get the date from the next sibling of the <b> tag
                    date = b_tag.next_sibling.strip() if b_tag else None
                else:
                    b_tag = section.find('b', text='от:')
                    # Get the date from the next sibling of the <b> tag
                    date = b_tag.next_sibling.strip() if b_tag else None
                
                if date == None:
                    # Find the <p> tag containing the date
                    date_tag = section.find('div', class_='heroes_card__nagrada-info')
                    if date_tag != None:
                        date_tag_p = date_tag.find('p', style="margin: 0;")
                        if date_tag_p != None:
                            # Get the date text
                            date = date_tag_p.get_text(strip=True) if date_tag_p else None
                awards.append([award_name, date])

            return awards
        
        finally:
            driver.quit()

# Example usage
awards = get_awards_by_date(url1)
print(awards)

https://pamyat-naroda.ru/heroes/person-hero123940112/?backurl=%2Fheroes%2F%3Fadv_search%3Dy%26last_name%3D%D0%91%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D1%8F%D0%BD%26first_name%3D%D0%98%D0%B2%D0%B0%D0%BD%26middle_name%3D%D0%A5%D1%80%D0%B8%D1%81%D1%82%D0%BE%D1%84%D0%BE%D1%80%D0%BE%D0%B2%D0%B8%D1%87%26group%3Dall%26types%3Dpamyat_commander%3Anagrady_nagrad_doc%3Anagrady_uchet_kartoteka%3Anagrady_ubilein_kartoteka%3Apdv_kart_in%3Apdv_kart_in_inostranec%3Apamyat_voenkomat%3Apotery_vpp%3Apamyat_zsp_parts%3Akld_ran%3Akld_bolezn%3Akld_polit%3Akld_upk%3Akld_vmf%3Akld_partizan%3Apotery_doneseniya_o_poteryah%3Apotery_gospitali%3Apotery_utochenie_poter%3Apotery_spiski_zahoroneniy%3Apotery_voennoplen%3Apotery_iskluchenie_iz_spiskov%3Apotery_kartoteki%3Apotery_rvk_extra%3Apotery_isp_extra%3Asame_doroga%3Asame_rvk%3Asame_guk%3Apotery_knigi_pamyati%26page%3D1%26grouppersons%3D1&
[['Орден Красного Знамени', '06.11.1941'], ['Орден Кутузова I степени', '30.08.1943'], ['Орден Кутузова I степени', '09.04.1943'], [

In [ ]:
#df9['awards_list'] = df9['link_PN'].apply(get_awards_selenium)

In [103]:
#df9.to_excel('intermediate_df9_with_awards.xls', index=False, engine='openpyxl')

## PARSER - GET AWARDS WITH DATES FROM EACH LINK TO "PAMYAT NARODA"

In [ ]:
df8 = pd.read_excel("intermediate_df8_with_PN.xls")
df8.head(3)

In [47]:
df8['awards_list_dates'] = df8['link_PN'].apply(get_awards_by_date)

https://pamyat-naroda.ru/heroes/person-hero123792421/?backurl=%2Fheroes%2F%3Fadv_search%3Dy%26last_name%3D%D0%90%D0%BA%D0%B8%D0%BC%D0%BE%D0%B2%26first_name%3D%D0%A1%D1%82%D0%B5%D0%BF%D0%B0%D0%BD%26middle_name%3D%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B5%D0%B2%D0%B8%D1%87%26date_birth_from%3D1896.0%26group%3Dall%26types%3Dpamyat_commander%3Anagrady_nagrad_doc%3Anagrady_uchet_kartoteka%3Anagrady_ubilein_kartoteka%3Apdv_kart_in%3Apdv_kart_in_inostranec%3Apamyat_voenkomat%3Apotery_vpp%3Apamyat_zsp_parts%3Akld_ran%3Akld_bolezn%3Akld_polit%3Akld_upk%3Akld_vmf%3Akld_partizan%3Apotery_doneseniya_o_poteryah%3Apotery_gospitali%3Apotery_utochenie_poter%3Apotery_spiski_zahoroneniy%3Apotery_voennoplen%3Apotery_iskluchenie_iz_spiskov%3Apotery_kartoteki%3Apotery_rvk_extra%3Apotery_isp_extra%3Asame_doroga%3Asame_rvk%3Asame_guk%3Apotery_knigi_pamyati%26page%3D1%26grouppersons%3D1&
https://pamyat-naroda.ru/heroes/person-hero123788555/?backurl=%2Fheroes%2F%3Fadv_search%3Dy%26last_name%3D%D0%90%D0%BD%D1%8

In [49]:
df8.to_excel('intermediate_df9_with_awards.xls', index=False, engine='openpyxl')

In [51]:
df8['link_PN'].isnull().sum()

41

In [53]:
df9 = pd.read_excel("intermediate_df9_with_awards.xls")
df9['unique_id'] = [uuid.uuid4() for _ in range(len(df9))]
df9.head(3)

,nationality,fullname,biography1,birth_year,biography1_source,link1,birth_place,link2,biography2,birth_year2,biography2_source,Pamyat_naroda_link,link_PN,awards_list_dates,unique_id
0,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,"[[None, '16.08.1936'], ['Орден Красной Звезды'...",3c18be5a-2781-4d82-a3df-0b25299b3965
1,белорус,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,"[['Орден Красного Знамени', '__.__.1922'], ['О...",12fd4813-e4a9-4060-8bd1-e525999333d1
2,армянин,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,"[['Орден Красного Знамени', '06.11.1941'], ['О...",8186645d-7137-43e6-8b7f-7b46543f5f62


In [54]:
df9['awards_list_dates'][0]

"[[None, '16.08.1936'], ['Орден Красной Звезды', '16.08.1936'], ['Орден Красного Знамени', '22.02.1938'], ['Орден Ленина', '07.04.1940'], ['Орден Красного Знамени', '25.07.1941'], [None, '25.07.1941'], [None, '25.07.1941'], ['Орден Красного Знамени', '25.07.1941'], ['Орден Красного Знамени', '25.07.1941']]"

## MAKE WIDE TO LONG DATAFRAME

In [55]:
# Function to safely evaluate the string representation
def safe_literal_eval(val):
    if val is None:
        return []
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        print(f"Error evaluating string: {val}")
        return []

# Apply the safe literal evaluation to the awards column
df9['awards_list_dates'] = df9['awards_list_dates'].apply(safe_literal_eval)

In [56]:
def expand_and_merge(df, id_col, awards_col):
    
    # Expand the DataFrame
    expanded_rows = []
    for _, row in df.iterrows():
        if row[awards_col]:
            for award in row[awards_col]:
                new_row = {id_col: row[id_col], 'award': award[0], 'date': award[1]}
                expanded_rows.append(new_row)
        else:
            # If there are no awards, still keep the original row
            new_row = {id_col: row[id_col], 'award': None, 'date': None}
            expanded_rows.append(new_row)

    expanded_df = pd.DataFrame(expanded_rows)

    # Merge the expanded DataFrame back with the original DataFrame, excluding the awards column
    merged_df = pd.merge(df.drop(columns=[awards_col]), expanded_df, on=id_col, how='left')
    
    return merged_df

expanded_merged_df = expand_and_merge(df9, 'unique_id', 'awards_list_dates')
expanded_merged_df

,nationality,fullname,biography1,birth_year,biography1_source,link1,birth_place,link2,biography2,birth_year2,biography2_source,Pamyat_naroda_link,link_PN,unique_id,award,date
0,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,3c18be5a-2781-4d82-a3df-0b25299b3965,None,16.08.1936
1,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,3c18be5a-2781-4d82-a3df-0b25299b3965,Орден Красной Звезды,16.08.1936
2,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,3c18be5a-2781-4d82-a3df-0b25299b3965,Орден Красного Знамени,22.02.1938
3,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,3c18be5a-2781-4d82-a3df-0b25299b3965,Орден Ленина,07.04.1940
4,русский,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Командармы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pamyat-naroda.ru/heroes/person-hero123...,3c18be5a-2781-4d82-a3df-0b25299b3965,Орден Красного Знамени,25.07.1941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11635,NaN,ЯСКИН Павел Николаевич,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",https://biografiaru.wordpress.com/%D0%B3%D0%B5...,NaN,https://biografiaru.wordpress.com/%D0%B3%D0%B5...,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",NaN,https://pamyat-naroda.ru/heroes/person-hero123...,055ca10a-5797-44e0-9abb-2989282cab77,None,
11636,NaN,ЯСКИН Павел Николаевич,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",https://biografiaru.wordpress.com/%D0%B3%D0%B5...,NaN,https://biografiaru.wordpress.com/%D0%B3%D0%B5...,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",NaN,https://pamyat-naroda.ru/heroes/person-hero123...,055ca10a-5797-44e0-9abb-2989282cab77,Орден Кутузова II степени,None
11637,NaN,ЯСКИН Павел Николаевич,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",https://biografiaru.wordpress.com/%D0%B3%D0%B5...,NaN,https://biografiaru.wordpress.com/%D0%B3%D0%B5...,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",NaN,https://pamyat-naroda.ru/heroes/person-hero123...,055ca10a-5797-44e0-9abb-2989282cab77,Медаль «За победу над Германией в Великой Отеч...,None
11638,NaN,ЯСКИН Павел Николаевич,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",https://biografiaru.wordpress.com/%D0%B3%D0%B5...,NaN,https://biografiaru.wordpress.com/%D0%B3%D0%B5...,ЯСКИН Павел Николаевич [23.12.1896 - 07.12.197...,1896.0,"biografiaru, centrasia",NaN,https://pamyat-naroda.ru/heroes/person-hero123...,055ca10a-5797-44e0-9abb-2989282cab77,Орден Ленина,None


In [57]:
expanded_merged_df.to_excel('df_long_awards.xls', index=False, engine='openpyxl')

In [58]:
expanded_merged_df['date'].isnull().sum()

2554